In [1]:
import tensorflow as tf

In [2]:
tf.__version__

'2.0.0'

In [3]:
v = tf.Variable(0.0)   #创建变量并赋值

In [4]:
(v+1).numpy()

1.0

In [5]:
v.assign(5)          #重新赋值
v

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=5.0>

In [6]:
v.assign_add(1)

<tf.Variable 'UnreadVariable' shape=() dtype=float32, numpy=6.0>

In [7]:
v.read_value()

<tf.Tensor: id=17, shape=(), dtype=float32, numpy=6.0>

自动微分运算 GradientTape()方法

In [8]:
w = tf.Variable([[1.0]])
with tf.GradientTape() as t:    #跟踪变量运算
    loss = w * w

In [9]:
grad = t.gradient(loss, w)    #自动微分，求解梯度
grad

<tf.Tensor: id=33, shape=(1, 1), dtype=float32, numpy=array([[2.]], dtype=float32)>

In [10]:
w = tf.constant(3.0)             #常量的跟踪
with tf.GradientTape() as t:    #跟踪变量运算
    t.watch(w)
    loss = w * w

In [11]:
dloss_dw = t.gradient(loss, w) 
dloss_dw

<tf.Tensor: id=39, shape=(), dtype=float32, numpy=6.0>

In [12]:
w = tf.constant(3.0)            
with tf.GradientTape(persistent=True) as t:       #gradient在调用后会立即释放空间，调用persistent参数可以是gradient永久保持
    t.watch(w)
    y = w * w
    z = y * y

In [13]:
dy_dw = t.gradient(y, w)
dy_dw

<tf.Tensor: id=46, shape=(), dtype=float32, numpy=6.0>

In [14]:
dz_dw = t.gradient(z, w)
dz_dw

<tf.Tensor: id=53, shape=(), dtype=float32, numpy=108.0>

# Example

In [15]:
(train_img, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

In [16]:
train_img.shape

(60000, 28, 28)

In [17]:
train_img = tf.expand_dims(train_img, -1)             #扩增维度expand_dims
train_img.shape

TensorShape([60000, 28, 28, 1])

In [18]:
test_images = tf.expand_dims(test_images, -1)             #扩增维度expand_dims
test_images.shape

TensorShape([10000, 28, 28, 1])

计算梯度只能对浮点类型计算

In [19]:
train_img = tf.cast(train_img/255, tf.float32)   #预处理

In [20]:
test_images = tf.cast(test_images/255, tf.float32)

In [21]:
train_labels = tf.cast(train_labels, tf.int64)
test_labels = tf.cast(test_labels, tf.int64)

In [22]:
dataset = tf.data.Dataset.from_tensor_slices((train_img, train_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))

In [23]:
dataset            #dataset数据集合并完成

<TensorSliceDataset shapes: ((28, 28, 1), ()), types: (tf.float32, tf.int64)>

In [24]:
dataset = dataset.shuffle(60000).batch(32)   #混洗，分批
test_dataset = test_dataset.batch(32)
dataset

<BatchDataset shapes: ((None, 28, 28, 1), (None,)), types: (tf.float32, tf.int64)>

In [25]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, [3, 3], activation = 'relu', input_shape = (None, None, 1)),
    tf.keras.layers.Conv2D(32, [3, 3], activation = 'relu'),
    tf.keras.layers.GlobalMaxPooling2D(),
    tf.keras.layers.Dense(10)   
])

不进行编译，自定义循环

In [26]:
optimizer = tf.keras.optimizers.Adam()   #调用优化函数

In [27]:
loss_func = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)    #大写的是可调用方法

In [28]:
features, labels = next(iter(dataset))         #dataset是可迭代数据

In [29]:
features.shape

TensorShape([32, 28, 28, 1])

In [30]:
labels.shape

TensorShape([32])

In [31]:
predictions = model(features)
predictions.shape

TensorShape([32, 10])

In [32]:
tf.argmax(predictions, axis=1)             #初始预测

<tf.Tensor: id=208, shape=(32,), dtype=int64, numpy=
array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2], dtype=int64)>

In [33]:
labels                                      #实际labels

<tf.Tensor: id=190, shape=(32,), dtype=int64, numpy=
array([9, 8, 7, 1, 4, 6, 6, 9, 3, 4, 2, 8, 1, 6, 5, 6, 7, 4, 9, 3, 6, 3,
       4, 1, 7, 8, 5, 7, 0, 7, 2, 1], dtype=int64)>

In [34]:
train_loss = tf.keras.metrics.Mean('train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('train_accuracy')

test_loss = tf.keras.metrics.Mean('test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('test_accuracy')

In [35]:
def loss(model, x, y):
    y_ = model(x)
    return loss_func(y, y_)                 #定义了model，代入数据x后，计算其损失值loss并返回

In [36]:
def train_step(model, images, labels):                               #一步优化过程
    with tf.GradientTape() as t:
        pred = model(images)
        loss_step = loss_func(labels, pred)
        
        #loss_step = loss(model, images, labels)                      #计算损失函数        
    grads = t.gradient(loss_step, model.trainable_variables)         #计算损失函数与可训练参数之间的梯度
    optimizer.apply_gradients(zip(grads, model.trainable_variables)) #优化函数应用梯度对可训练参数进行优化
    train_loss(loss_step)
    train_accuracy(labels, pred)


In [37]:
def test_step(model, images, labels):                          
    pred = model(images)
    loss_step = loss_func(labels, pred)
    test_loss(loss_step)
    test_accuracy(labels, pred)


In [38]:
def train():
    for epoch in range(10):
        for (batch, (images, labels)) in enumerate(dataset):
            train_step(model, images, labels)
        print('epoch{} loss is {}, train_accuracy is {}'.format(epoch, train_loss.result(), train_accuracy.result()))
        
        for (batch, (images, labels)) in enumerate(test_dataset):
            test_step(model, images, labels)
        print('epoch{} loss is {}, test_accuracy is {}'.format(epoch, test_loss.result(), test_accuracy.result()))
        
        train_loss.reset_states()
        train_accuracy.reset_states()
        test_loss.reset_states()
        test_accuracy.reset_states()

In [39]:
train()

epoch0 loss is 0.8186440467834473, train_accuracy is 0.751466691493988
epoch0 loss is 0.39579376578330994, test_accuracy is 0.8798999786376953
epoch1 loss is 0.37636858224868774, train_accuracy is 0.8813666701316833
epoch1 loss is 0.3063313961029053, test_accuracy is 0.9053000211715698
epoch2 loss is 0.3120986223220825, train_accuracy is 0.9014333486557007
epoch2 loss is 0.2628791928291321, test_accuracy is 0.9205999970436096
epoch3 loss is 0.27545177936553955, train_accuracy is 0.9132500290870667
epoch3 loss is 0.24120287597179413, test_accuracy is 0.9232000112533569
epoch4 loss is 0.2506524622440338, train_accuracy is 0.9206500053405762
epoch4 loss is 0.2215137630701065, test_accuracy is 0.9314000010490417
epoch5 loss is 0.2327248901128769, train_accuracy is 0.925849974155426
epoch5 loss is 0.20930048823356628, test_accuracy is 0.9345999956130981
epoch6 loss is 0.2204141914844513, train_accuracy is 0.9298166632652283
epoch6 loss is 0.20640751719474792, test_accuracy is 0.937200009822

# tf.keras.metrics 汇总计算模块

In [40]:
m = tf.keras.metrics.Mean('acc')     #调用汇总计算模块

In [41]:
m(10)

<tf.Tensor: id=2741703, shape=(), dtype=float32, numpy=10.0>

In [42]:
m(20)

<tf.Tensor: id=2741713, shape=(), dtype=float32, numpy=15.0>

In [43]:
m.result().numpy()            #使用result方法返回所有计算对象的均值

15.0

In [44]:
m([30,40])

<tf.Tensor: id=2741727, shape=(), dtype=float32, numpy=25.0>

In [45]:
m.result().numpy()

25.0

In [46]:
m.reset_states()               #重置
m.result().numpy()

0.0

In [47]:
a = tf.keras.metrics.SparseCategoricalAccuracy('acc')

In [48]:
a(labels, model(features))

<tf.Tensor: id=2741781, shape=(), dtype=float32, numpy=1.0>